In [1]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

from Models.ActorCritic import ActorCritic
from Models.SoftActorCritic import SoftActorCritic
from Models.DDQN import DoubleDQN

pygame 2.3.0 (SDL 2.24.2, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\kaush\AppData\Local\Temp\ipykernel_15444\2412810607.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
torch.cuda.is_available()

True

In [3]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [4]:
env = CustomEnvironment(env_config=env_variables, render_mode="human")
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [5]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 128))

In [6]:
ac_config = open('Environment/ac_config.json')
ac_variables = json.load(ac_config)
ac_config.close

sac_config = open('Environment/sac_config.json')
sac_variables = json.load(sac_config)
sac_config.close

ddqn_config = open('Environment/ddqn_config.json')
ddqn_variables = json.load(ddqn_config)
ddqn_config.close

<function TextIOWrapper.close()>

In [7]:
ddqn = DoubleDQN(ddqn_variables)
# ac = ActorCritic(ac_variables)
# sac = SoftActorCritic(sac_variables)

In [8]:
models = {0:ddqn, 1:ddqn, 2:ddqn}

In [9]:
episodes = 400

for episode in tqdm_notebook(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",
    # print(curr_obs[0][1])

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for a in env.agents:
            # print(curr_obs[a])
            act, log_prob = models[a].select_action(state = torch.tensor(curr_obs[a], dtype = torch.float32))
            actions[a] = act
            log_probs[a] = log_prob

        # for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
        #     if i in env.agents:
        #         act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
        #         actions[i] = act
        #         log_probs[i] = log_prob
        #     else:
        #         actions[i] = env.num_actions
        #         log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        if env.timestep % 50 == 0:
            print(rewards)
        
        # print(next_obs)
        # print(env.agents_objects)
        # print(curr_obs)
        # print(rewards)
        # print(terminations)
        # print(truncations)
        # print(infos)

        for id, obs in next_obs.items():
            models[id].push_to_buffer(torch.tensor(curr_obs[id], dtype = torch.float32),actions[id],rewards[id],torch.tensor(obs, dtype = torch.float32),log_probs[id],terminations[id])
            loss = models[id].update_weights()

            if(terminations[id] == True):
                # loss = models[id].update_weights()
                losses[id] = loss
        
        # for i in range(env.n_agents):

        #     if i in env.agents:
        #         env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         env.agents_objects[i].brain1.update_weights() 
            
        #     if i in env.justdie:
        #         env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         loss = env.justdie[i].brain1.update_weights() 
        #         del env.justdie[i]
        #         losses[i] = loss
        
        curr_obs = next_obs

    print(f"Episode {episode} end")
    print(f"Losses {losses}")
    print()
    env.close()

  0%|          | 0/400 [00:00<?, ?it/s]

{0: -0.19810583925260752, 1: -0.29998375992969073, 2: -0.19485350753254888}
agent2 died at time 99
{0: -0.4432375626540308, 1: -0.3337267359933387, 2: -0.32269810217418626}
agent1 died at time 141
{0: -0.21824584594350172}
agent0 died at time 162
Episode 0 end
Losses {2: tensor(4.0715), 1: tensor(14.1741), 0: tensor(18.9862)}

{0: -0.1950462004896739, 1: -0.5017602399977099, 2: -0.3851883315397996}
{0: -0.5077881312062971, 1: -0.3411888099640852, 2: -0.27154267986459196}
agent0 died at time 120
agent1 died at time 120
agent2 died at time 141
Episode 1 end
Losses {0: tensor(43.7662), 1: tensor(41.0439), 2: tensor(37.2333)}

{0: -0.42538683605738375, 1: -0.3411693844543208, 2: -0.3746019903499708}
{0: -0.3351513609666754, 1: -0.5421317268930497, 2: -0.2801491156819956}
agent0 died at time 120
agent2 died at time 120
{1: -0.49006807158677096}
agent1 died at time 162
Episode 2 end
Losses {0: tensor(22.5497), 2: tensor(16.1040), 1: tensor(16.4637)}

{0: -0.3577383516379395, 1: -0.3278174781

In [ ]:
models[0].steps_done